## Data Training

In [ ]:
import digitalhub as dh

project = dh.get_or_create_project("project-mlflow-model-ci")

In [ ]:
import os
directory="src"
if not os.path.exists(directory):
    os.makedirs(directory)

In [ ]:
%%writefile "src/train-model.py"

from digitalhub_runtime_python import handler
from digitalhub import from_mlflow_run
import mlflow

from sklearn import datasets, svm
from sklearn.model_selection import GridSearchCV

@handler(outputs=["model"])
def train(project):
    mlflow.sklearn.autolog(log_datasets=True)

    iris = datasets.load_iris()
    parameters = {"kernel": ("linear", "rbf"), "C": [1, 10]}
    svc = svm.SVC()
    clf = GridSearchCV(svc, parameters)

    clf.fit(iris.data, iris.target)
    run_id = mlflow.last_active_run().info.run_id

    # utility to map mlflow run artifacts to model metadata
    model_params = from_mlflow_run(run_id)

    return (project.log_model(
        name="model-mlflow",
        kind="mlflow",
        **model_params
))

In [ ]:
train_fn = project.new_function(name="train",
                                kind="python",
                                python_version="PYTHON3_10",
                                code_src="src/train-model.py",
                                handler="train",
                                requirements=["scikit-learn", "mlflow"])

In [ ]:
train_model_run = train_fn.run(action="job", local_execution=False)

Wait until the train model job is completed. One can view the state of run from console or using digitalhub sdk API using

In [ ]:
train_model_run.refresh().status.state

Once 'Completed', proceed with model 'serve' step.

## Serve

In [ ]:
model_path=train_model_run.refresh().output('model').spec.path
model_path

In [ ]:
func = project.new_function(name="serve_mlflowmodel",
                            kind="mlflowserve",
                            model_name="mlflow_model",
                            path=model_path + 'model/')

In [ ]:
serve_run = func.run(action="serve", wait=True)

Let us wait till the 'serve' is loaded compeletey

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()
data = iris.data[0:2].tolist()
json={
    "inputs": [
        {
        "name": "input-0",
        "shape": [-1, 4],
        "datatype": "FP64",
        "data": data
        }
    ]
}

In [ ]:
json

In [ ]:
service = serve_run.refresh().status.service
service

Once 'Service' object available, proceed by making call to deployed service.

In [ ]:
service_url = serve_run.refresh().status.to_dict()["service"]["url"]
url = f"http://{service_url}/v2/models/mlflow_model/infer"
url

<!-- /v2/models/model/infer -->

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()
data = iris.data[0:2].tolist()
json={
    "inputs": [
        {
        "name": "input-0",
        "shape": [-1, 4],
        "datatype": "FP64",
        "data": data
        }
    ]
}

json

In [ ]:
serve_run.invoke(url=url, json=json).json()
# serve_run.invoke(model_name="mlflow_model", url=url, json=json).json()

## Pipeline

In [ ]:
%%writefile "src/mlflow_pipeline.py"

from digitalhub_runtime_kfp.dsl import pipeline_context

def myhandler(): 
    with pipeline_context() as pc:
        train = pc.step(name="train", function="train", action="job")
        #serve = pc.step(name="serve", function="serve_mlflowmodel", action="serve")

In [ ]:
workflow = project.new_workflow(name="pipeline_mlflow", kind="kfp", code_src= "src/mlflow_pipeline.py", handler = "myhandler")

In [ ]:
wf_run = workflow.run()

In [ ]:
wf_run.refresh().status.state